# Can You Forecast Daily New Covid Cases in Texas?

## The Datasets
* The data sets include daily new covid case count curves for TX, NY, and NJ:
 * **TX_Corona_Curve.csv** - 06/19/2020 - 07/06/2020, 18 days data, incomplete curve.
 * **NY_Corona_Curve.csv** - 03/20/2020 - 05/05/2020, 48 days data, completed curve.
 * **NJ_Corona_Curve.csv** - 03/26/2020 - 05/18/2020, 54 days data, completed curve.
 * **Corona_Curves_TX_NY_NJ.csv** - All the data above combined and normalized using each state's population.  
 
* This data was painfully collected from the daily new case counts graphs at: 
 * https://www.worldometers.info/coronavirus/usa/texas/
 * https://www.worldometers.info/coronavirus/usa/new-york/
 * https://www.worldometers.info/coronavirus/usa/new-jersey/

## Data Definitions
* **Pop_Pct** - Represents the % of each respective state's population for the daily new case count (daily new cases / state population)
* **Curve_Day** - Curve day 1 starts when the daily new case count rises above a 0.00015 case to state population ratio.  The curve finishes when the daily new case count falls below a 0.00015 case to state population ratio.    
* **Cases** - The number of daily new Covid Cases for each state. The raw counts are only in the state level files. 
* **Pct_Change** - The change in this percentage for the prior day. (Pop_Pct - Previous day's Pop_Pct) 
* **M1 thru M7** - The prior value minus 1 to minus 7 days ago. 
* **Three_Day_Avg** - The 3 day average including today's value.
* **Seven_Day_Avg** - The 7 day average including today's value.

## NOTE!
**This notebook is not intended to show you the best way to forecast the curve, but rather to introduce you to a few tools that may help you do so.**

In [20]:
#import required Libraries
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose

import chart_studio.plotly as plt

In [2]:
df_corona = pd.read_csv("../../data/Corona_Curves_TX_NY_NJ.csv", low_memory=False)
df_corona["Date"] = pd.to_datetime(df_corona["Date"])
df_corona.sort_values(by=['Date'], inplace=True)


In [3]:
df_corona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119 entries, 18 to 17
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   State                  119 non-null    object        
 1   Date                   119 non-null    datetime64[ns]
 2   Daily_New_Cases        119 non-null    int64         
 3   Curve_Day              119 non-null    int64         
 4   Pop_Pct                119 non-null    float64       
 5   Pop_Pct_M1             119 non-null    float64       
 6   Pop_Pct_M2             119 non-null    float64       
 7   Pop_Pct_M3             119 non-null    float64       
 8   Pop_Pct_M4             119 non-null    float64       
 9   Pop_Pct_M5             119 non-null    float64       
 10  Pop_Pct_M6             119 non-null    float64       
 11  Pop_Pct_M7             119 non-null    float64       
 12  Three_Day_Avg_Pop_Pct  119 non-null    float64       
 13  Seven

## What does the data look like?

In [5]:
pd.set_option('display.max_rows', 500)
df_corona = df_corona.reset_index()
df_corona.head()

,index,State,Date,Daily_New_Cases,Curve_Day,Pop_Pct,Pop_Pct_M1,Pop_Pct_M2,Pop_Pct_M3,Pop_Pct_M4,...,Pct_Change,Pct_Change_M1,Pct_Change_M2,Pct_Change_M3,Pct_Change_M4,Pct_Change_M5,Pct_Change_M6,Pct_Change_M7,Three_Day_Avg_Pct_Chg,Seven_Day_Avg_Pct_Chg
0,18,NY,2020-03-20,3052,1,0.000162,0.000124,0.000071,0.000039,0.000012,...,0.000038,0.000053,0.000032,0.000027,0.000001,0.000006,7.450000e-07,0.000000e+00,0.00004,0.00002
1,19,NY,2020-03-21,1993,2,0.000106,0.000162,0.000124,0.000071,0.000039,...,-0.000056,0.000038,0.000053,0.000032,0.000027,0.000001,5.584000e-06,7.450000e-07,0.00001,0.00001
2,20,NY,2020-03-22,5440,3,0.000289,0.000106,0.000162,0.000124,0.000071,...,0.000183,-0.000056,0.000038,0.000053,0.000032,0.000027,1.223000e-06,5.584000e-06,0.00005,0.00004
3,21,NY,2020-03-23,5123,4,0.000272,0.000289,0.000106,0.000162,0.000124,...,-0.000017,0.000183,-0.000056,0.000038,0.000053,0.000032,2.696200e-05,1.223000e-06,0.00004,0.00004
4,22,NY,2020-03-24,5516,5,0.000293,0.000272,0.000289,0.000106,0.000162,...,0.000021,-0.000017,0.000183,-0.000056,0.000038,0.000053,3.190800e-05,2.696200e-05,0.00006,0.00004


In [37]:
# 
fig = px.line(df_corona, x="Date", y="Daily_New_Cases", title='Daily New Cases', color='State')
fig.show()

In [39]:
# 
fig = px.line(df_corona, x="Date", y="Pop_Pct", title='Daily New Cases (% of Population)', color='State')
fig.show()

## Create a Test and Train Dataset

In [5]:
y = corona_data["Pop_Pct"]
X = corona_data.drop(['index','State','Date','Pop_Pct','Daily_New_Cases','Pct_Change'], 1)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Curve_Day              119 non-null    int64  
 1   Pop_Pct_M1             119 non-null    float64
 2   Pop_Pct_M2             119 non-null    float64
 3   Pop_Pct_M3             119 non-null    float64
 4   Pop_Pct_M4             119 non-null    float64
 5   Pop_Pct_M5             119 non-null    float64
 6   Pop_Pct_M6             119 non-null    float64
 7   Pop_Pct_M7             119 non-null    float64
 8   Three_Day_Avg_Pop_Pct  119 non-null    float64
 9   Seven_Day_Avg_Pop_Pct  119 non-null    float64
 10  Pct_Change_M1          119 non-null    float64
 11  Pct_Change_M2          119 non-null    float64
 12  Pct_Change_M3          119 non-null    float64
 13  Pct_Change_M4          119 non-null    float64
 14  Pct_Change_M5          119 non-null    float64
 15  Pct_Ch

## Perform Cross Validation for Time Series Data

### Time Series Split
**This is a form of cross validation for time series data where each fold is a superset of the previous fold, and test records are selected from records / indices which occur after the training data.**
* Note below that each 
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [6]:
from sklearn.model_selection import TimeSeriesSplit

n=10
cv = TimeSeriesSplit(n_splits=n)
x = cv.split(X,y)

for i in range(n):
    train, test = next(x)
    print('Fold',i,'Train Records:',len(train),'Test Records:', len(test))

Fold 0 Train Records: 19 Test Records: 10
Fold 1 Train Records: 29 Test Records: 10
Fold 2 Train Records: 39 Test Records: 10
Fold 3 Train Records: 49 Test Records: 10
Fold 4 Train Records: 59 Test Records: 10
Fold 5 Train Records: 69 Test Records: 10
Fold 6 Train Records: 79 Test Records: 10
Fold 7 Train Records: 89 Test Records: 10
Fold 8 Train Records: 99 Test Records: 10
Fold 9 Train Records: 109 Test Records: 10


## Scale the Dataset

In [7]:
# Scale dataset converting to standard normally distributed data 
# (e.g. Gaussian with 0 mean and unit variance).
from sklearn.preprocessing import StandardScaler

#Fit to data for scaling
scaler = StandardScaler()
scaler.fit(X)

# Transform training data
# This makes our model's coefficients take on the same scale for 
# accurate feature importance analisys 
X_scaled = scaler.transform(X)

X_scaled.shape

(119, 19)

### Polynomial Features 
**You may use polynomial features to perform regression on curved data**

#### Helpful Links
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html
* https://stackoverflow.com/questions/51906274/cannot-understand-with-sklearns-polynomialfeatures
* https://stackoverflow.com/questions/32660231/how-to-fit-a-polynomial-curve-to-data-using-scikit-learn

In [8]:
# Create polynomial features to try and let the model know we are dealing with a curve
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
poly.fit(X_scaled)

X_scaled_poly = poly.transform(X_scaled)

X_scaled_poly.shape

(119, 210)

## Helper Functions and Metrics to Evaluate our Regression Model Performance  

In [9]:
#Use mean absolute error (MAE) to score the regression models created 
#(the scale of MAE is identical to the response variable)
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

#Function for Mean Absolute Percentage Error (MAPE) - Untested
#Adapted from - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python
def mape(y_actual, y_predicted): 
    mask = y_actual != 0
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100

#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
mape_scorer = make_scorer(score_func=mape, greater_is_better=False)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
                'MAPE': mape_scorer
               } 

In [10]:
from sklearn.model_selection import cross_validate

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

    #print mean test_MAPE for all folds
    scores['test_MAPE'] = scores['test_MAPE']
    mape_avg = scores['test_MAPE'].mean()
    print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
    print(print_str.format(mape_avg=mape_avg))

    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    return scoresResults

## Linear Regression Validation

In [11]:
from sklearn.linear_model import LinearRegression

#Create a Linear Regression object and perform a grid search to find the best parameters
linreg = LinearRegression()
parameters = {'normalize':(True,False), 'fit_intercept':(True,False)}

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_scaled_poly, y)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             estimator=LinearRegression(),
             param_grid={'fit_intercept': (True, False),
                         'normalize': (True, False)},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False),
             verbose=1)

In [12]:
#Print the parameterization of the best estimator
regGridSearch.best_estimator_

LinearRegression()

In [13]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
#pd.set_option('display.float_format', '{:.17f}'.format)
pd.reset_option('display.float_format')
EvaluateRegressionEstimator(regEstimator, X_scaled_poly, y, cv)

The average MAE for all cv folds is: 			 4.0302e-05
The average MAE percentage (MAPE) for all cv folds is: 	 12.912
The average RMSE for all cv folds is: 			 5.0675e-05
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.000076,15.702509,0.000091
1,0.000023,5.304188,0.000033
2,0.000105,26.965971,0.000123
3,0.000043,12.120289,0.000058
4,0.000031,7.491286,0.000039
5,0.000080,36.736664,0.000109
6,0.000028,13.984221,0.000033
7,0.000006,4.496953,0.000008
8,0.000008,4.769978,0.000009
9,0.000003,1.544798,0.000004


In [14]:
# Convert the Population Percent MAE (Mean Absolute Error)
# To a Daily New Cases MAE using the Texas Population 
Texas_Population = 29900000
print('Overall Case MAE')
print(Texas_Population * 4.0174e-05)
print(" ")
print('Case MAE by Fold')
print(0," Case MAE:",Texas_Population * 0.000076)
print(1," Case MAE:",Texas_Population * 0.000023)
print(2," Case MAE:",Texas_Population * 0.000106)
print(3," Case MAE:",Texas_Population * 0.000044)
print(4," Case MAE:",Texas_Population * 0.000030)
print(5," Case MAE:",Texas_Population * 0.000078)
print(6," Case MAE:",Texas_Population * 0.000028)
print(7," Case MAE:",Texas_Population * 0.000006)
print(8," Case MAE:",Texas_Population * 0.000008)
print(9," Case MAE:",Texas_Population * 0.000003)

Overall Case MAE
1201.2025999999998
 
Case MAE by Fold
0  Case MAE: 2272.4
1  Case MAE: 687.7
2  Case MAE: 3169.4
3  Case MAE: 1315.6
4  Case MAE: 897.0
5  Case MAE: 2332.2
6  Case MAE: 837.1999999999999
7  Case MAE: 179.4
8  Case MAE: 239.2
9  Case MAE: 89.7


## Lasso Regression Validation

In [15]:
#Create a regression object and perform a grid search to find the best parameters
from sklearn.linear_model import Lasso

reg = Lasso(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=10000, precompute=True, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 10, 20]
selection = ['cyclic','random']
warm_start = [True, False]
parameters = {'alpha': alpha, 'selection': selection, 'warm_start': warm_start}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=-1 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_scaled_poly, y)

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    1.6s finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             estimator=Lasso(max_iter=10000, normalize=True, precompute=True,
                             random_state=0),
             n_jobs=-1,
             param_grid={'alpha': [0.001, 0.1, 1, 10, 20],
                         'selection': ['cyclic', 'random'],
                         'warm_start': [True, False]},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False),
             verbose=1)

In [16]:
#Print the parameterization of the best estimator
regGridSearch.best_estimator_

Lasso(alpha=0.001, max_iter=10000, normalize=True, precompute=True,
      random_state=0, warm_start=True)

In [17]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
#pd.set_option('display.float_format', '{:.17f}'.format)
pd.reset_option('display.float_format')
EvaluateRegressionEstimator(regEstimator, X_scaled_poly, y, cv)

The average MAE for all cv folds is: 			 0.00011749
The average MAE percentage (MAPE) for all cv folds is: 	 54.243
The average RMSE for all cv folds is: 			 0.00013243
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.000167,34.050261,0.000183
1,0.000090,17.437702,0.000125
2,0.000086,23.410746,0.000107
3,0.000058,19.198983,0.000073
4,0.000073,20.088855,0.000094
5,0.000122,52.827885,0.000132
6,0.000134,69.114829,0.000145
7,0.000203,163.742139,0.000208
8,0.000153,96.096449,0.000157
9,0.000088,46.461249,0.000101


In [18]:
# Convert the Population Percent MAE (Mean Absolute Error)
# To a Daily New Cases MAE using the Texas Population 
Texas_Population = 29900000
print('Overall Case MAE')
print(Texas_Population * 0.00011749)
print(" ")
print('Case MAE by Fold')
print(0," Case MAE:",Texas_Population * 0.000167)
print(1," Case MAE:",Texas_Population * 0.000090)
print(2," Case MAE:",Texas_Population * 0.000086)
print(3," Case MAE:",Texas_Population * 0.000058)
print(4," Case MAE:",Texas_Population * 0.000073)
print(5," Case MAE:",Texas_Population * 0.000122)
print(6," Case MAE:",Texas_Population * 0.000134)
print(7," Case MAE:",Texas_Population * 0.000203)
print(8," Case MAE:",Texas_Population * 0.000153)
print(9," Case MAE:",Texas_Population * 0.000088)

Overall Case MAE
3512.951
 
Case MAE by Fold
0  Case MAE: 4993.3
1  Case MAE: 2691.0
2  Case MAE: 2571.4
3  Case MAE: 1734.2
4  Case MAE: 2182.7
5  Case MAE: 3647.7999999999997
6  Case MAE: 4006.6
7  Case MAE: 6069.7
8  Case MAE: 4574.7
9  Case MAE: 2631.2


## Random Forest Regression Validation

In [19]:
#Create a Linear Regression object and perform a grid search to find the best parameters
from sklearn.ensemble import RandomForestRegressor

linreg = RandomForestRegressor()
parameters = { 'min_samples_split':[2,3,4,5]
              ,'n_estimators' : [500]
              ,'min_samples_leaf': [10, 25, 50]
              ,'criterion': ['mae']
              ,'n_jobs':[8] 
              ,'random_state': [0]
             }

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , n_jobs=-1 
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_scaled_poly, y)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   46.7s finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
             estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['mae'], 'min_samples_leaf': [10, 25, 50],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [500], 'n_jobs': [8],
                         'random_state': [0]},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False),
             verbose=1)

In [20]:
#Print the parameterization of the best estimator
regGridSearch.best_estimator_

RandomForestRegressor(criterion='mae', min_samples_leaf=10, min_samples_split=3,
                      n_estimators=500, n_jobs=8, random_state=0)

In [21]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
#pd.set_option('display.float_format', '{:.17f}'.format)
pd.reset_option('display.float_format')
EvaluateRegressionEstimator(regEstimator, X_scaled_poly, y, cv)

The average MAE for all cv folds is: 			 0.00010007
The average MAE percentage (MAPE) for all cv folds is: 	 45.194
The average RMSE for all cv folds is: 			 0.0001153
*********************************************************
Cross Validation Fold Mean Error Scores


,MAE,MAPE,RMSE
0,0.000164,33.426906,0.000181
1,0.000082,15.995336,0.000116
2,0.000077,21.397830,0.000096
3,0.000049,15.720081,0.000062
4,0.000064,17.517132,0.000083
5,0.000107,46.530357,0.000116
6,0.000127,66.819020,0.000142
7,0.000166,136.597222,0.000173
8,0.000100,63.669425,0.000107
9,0.000063,34.263122,0.000077


In [22]:
# Convert the Population Percent MAE (Mean Absolute Error)
# To a Daily New Cases MAE using the Texas Population 
Texas_Population = 29900000
print('Overall Case MAE')
print(Texas_Population * 9.7141e-05)
print(" ")
print('Case MAE by Fold')
print(0," Case MAE:",Texas_Population * 0.000164)
print(1," Case MAE:",Texas_Population * 0.000081)
print(2," Case MAE:",Texas_Population * 0.000075)
print(3," Case MAE:",Texas_Population * 0.000057)
print(4," Case MAE:",Texas_Population * 0.000062)
print(5," Case MAE:",Texas_Population * 0.000097)
print(6," Case MAE:",Texas_Population * 0.000112)
print(7," Case MAE:",Texas_Population * 0.000164)
print(8," Case MAE:",Texas_Population * 0.000106)
print(9," Case MAE:",Texas_Population * 0.000054)

Overall Case MAE
2904.5159
 
Case MAE by Fold
0  Case MAE: 4903.6
1  Case MAE: 2421.9
2  Case MAE: 2242.5
3  Case MAE: 1704.3000000000002
4  Case MAE: 1853.8000000000002
5  Case MAE: 2900.3
6  Case MAE: 3348.7999999999997
7  Case MAE: 4903.6
8  Case MAE: 3169.4
9  Case MAE: 1614.6


# Create Synthetic Data Using Predictions from the Model

### Create Starting Datasets for Simulation

In [23]:
#import required Libraries
import pandas as pd
import numpy as np

# Load the corona curves dataset
corona_data = pd.read_csv("../../data/Corona_Curves_TX_NY_NJ.csv",low_memory=False)
corona_data["Date"] = pd.to_datetime(corona_data["Date"])
corona_data.sort_values(by=['Date'], inplace=True)

# Create the TX curve dataset 
# (we add the simulation data to this later)
tx_curve = corona_data[corona_data['State'] == 'TX'].copy()

print('Starting Training Dataset (rows, columns)')
print('----------------------------------------------------------')
corona_data.info(verbose=False)
print(' ')
print('Starting Texas Dataset')
print('----------------------------------------------------------')
tx_curve.info(verbose=False)

FileNotFoundError: [Errno 2] File ./../data/Corona_Curves_TX_NY_NJ.csv does not exist: './../data/Corona_Curves_TX_NY_NJ.csv'

### Run Simulation Using the Model

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import datetime

# Retrain the model using the prior day's synthetic data
reg = Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=True, positive=False, precompute=True, random_state=0,
      selection='cyclic', tol=0.0001, warm_start=True)

scaler = StandardScaler()
poly = PolynomialFeatures(4)
Texas_Population = 29900000

for curve_day in range(18,55):
    # create the training dataset using the most recent synthetic data for each iteration
    y_syn = corona_data["Pop_Pct"]
    X_syn = corona_data.drop(['State','Date','Pop_Pct','Daily_New_Cases','Pct_Change'], 1)
    # Scale tarining dataset and create polynomial features (bends our regression line into a curve)
    X_syn_scaled = scaler.fit_transform(X_syn)
    X_syn_scaled_poly = poly.fit_transform(X_syn_scaled)
    # Fit model with the most recent synthetic training data
    reg.fit(X_syn_scaled_poly,y_syn)
    
    # Get the most recent curve day from the dataset
    prev_day = tx_curve[tx_curve['Curve_Day'] == curve_day].copy()
    
    # Now make the synthetic data required to get a prediction from the model for the next day  
    cur_day = tx_curve[tx_curve['Curve_Day'] == curve_day].copy()
    
    # Create the synthetic data for the curve day
    cur_day['Curve_Day'] = cur_day['Curve_Day'] + 1
    
    # Create the synthetic data for Pop_Pct
    cur_day['Pop_Pct_M7'] = cur_day['Pop_Pct_M6']
    cur_day['Pop_Pct_M6'] = cur_day['Pop_Pct_M5']
    cur_day['Pop_Pct_M5'] = cur_day['Pop_Pct_M4']
    cur_day['Pop_Pct_M4'] = cur_day['Pop_Pct_M3']
    cur_day['Pop_Pct_M3'] = cur_day['Pop_Pct_M2']
    cur_day['Pop_Pct_M2'] = cur_day['Pop_Pct_M1']
    cur_day['Pop_Pct_M1'] = cur_day['Pop_Pct']
    
    # Create the synthetic data for Pop_Pct averages
    cur_day['Three_Day_Avg_Pop_Pct'] = cur_day[['Pop_Pct_M1','Pop_Pct_M2','Pop_Pct_M3']].mean(axis=1) 
    cur_day['Seven_Day_Avg_Pop_Pct'] = cur_day[['Pop_Pct_M1','Pop_Pct_M2','Pop_Pct_M3',
                                                'Pop_Pct_M4','Pop_Pct_M5','Pop_Pct_M6','Pop_Pct_M7']].mean(axis=1) 
    
    # Create the synthetic data for Pct_Change
    cur_day['Pct_Change_M7'] = cur_day['Pct_Change_M6']
    cur_day['Pct_Change_M6'] = cur_day['Pct_Change_M5']
    cur_day['Pct_Change_M5'] = cur_day['Pct_Change_M4']
    cur_day['Pct_Change_M4'] = cur_day['Pct_Change_M3']
    cur_day['Pct_Change_M3'] = cur_day['Pct_Change_M2']
    cur_day['Pct_Change_M2'] = cur_day['Pct_Change_M1']
    cur_day['Pct_Change_M1'] = cur_day['Pct_Change']
    
    # Create the synthetic data for Pct_Change averages
    cur_day['Three_Day_Avg_Pct_Chg'] = cur_day[['Pct_Change_M1','Pct_Change_M2','Pct_Change_M3']].mean(axis=1) 
    cur_day['Seven_Day_Avg_Pct_Chg'] = cur_day[['Pct_Change_M1','Pct_Change_M2','Pct_Change_M3',
                                                'Pct_Change_M4','Pct_Change_M5','Pct_Change_M6','Pct_Change_M7']].mean(axis=1)
    
    #Make cur_day_X and scale
    cur_day_X = cur_day.drop(['State','Date','Pop_Pct','Daily_New_Cases','Pct_Change'], 1)
    cur_day_X_scaled = scaler.transform(cur_day_X)
    cur_day_X_poly = poly.transform(cur_day_X_scaled)
    
    #Get the predicted Pop_Pct using the model
    cur_day_pop_pct = reg.predict(cur_day_X_poly)
    
    #Finish creating the data for this day's record
    cur_day['Date'] = cur_day['Date'] + datetime.timedelta(days=1)
    cur_day['Pop_Pct'] = cur_day_pop_pct
    cur_day['Pct_Change'] = cur_day['Pop_Pct'] - prev_day['Pct_Change'] 
    cur_day['Daily_New_Cases'] = np.rint(cur_day['Pop_Pct'] * Texas_Population).astype(int)
    
    # Append the current day's predicted data to the Texas curve simulation output
    tx_curve = tx_curve.append(cur_day)
    # Append the current day's predicted data to our training dataset as a feedback loop
    corona_data = corona_data.append(cur_day)

NameError: name 'tx_curve' is not defined

### What do our synthetic data predictions look like?
**Currently each of the models tested seem to perform reasonable during cross validation, but do a poor job of making a curve.**

### Can you do it better???

In [25]:
pd.set_option('display.max_columns', 500)
tx_curve

NameError: name 'tx_curve' is not defined